In [26]:
import os
import pandas as pd
import numpy as np
import datetime

In [27]:
#Switch to local director with csvs
os.chdir('/Users/matthoffman/Repos/Weed/price-of-weed')

In [28]:
#Pull list of all CSV files; note: this could be refactored with glob

files = pd.Series(os.listdir('data'))
boolean = ['.csv' in x for x in files]
csvfiles = files[boolean].values

def extract_date(filename):
    return datetime.date(int(filename[14:18]), int(filename[12:14]), int(filename[10:12]))

def concat_dfs(filenames):
    files = []
    for filename in filenames:
        df = pd.read_csv('data/' + filename)
        df['Date'] = extract_date(filename)
        df.set_index(['State', 'Date'], inplace=True)
        files.append(df)
    return pd.concat(files)

def destringify(df,cols):
    for col in cols:
        df.ix[:,col] = [float(str(x).replace('$','')) for x in df[col].values]

In [29]:
df = concat_dfs(csvfiles)

In [30]:
#Remove State and Date indexes until can figure out how to groupby with multiindex (easy once it's multiindexed)
df.reset_index(inplace=True)

In [31]:
df['LowQ'].replace('I feel bad for these guys -->', np.nan, inplace=True)

In [32]:
#Remove occasional $s
destringify(df,['HighQ','MedQ','LowQ'])

In [33]:
#Use median to prevent outlier influence
avgstateprices = df.groupby('State').aggregate(np.median)

In [34]:
#Adjust for odd outlier to round down state with more expensive low quality weed to the most expensive state that seems "reasonable"
avgstateprices.sort(columns='LowQ',ascending=False)
avgstateprices.loc[['Montana','New Hampshire','Alaska'],'LowQ'] = avgstateprices.loc['North Dakota','LowQ']

In [35]:
os.chdir('/Users/matthoffman/Repos/Weed/Price-of-Weed-D3-Map')
avgstateprices.to_csv('Average Prices.csv')